Deprecated<br>
<br>
GitHub<br>
https://github.com/microsoft/GLIP<br>
論文<br>
https://arxiv.org/abs/2206.05836<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/GLIP_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## downgrade python version

In [ ]:
!python --version

In [ ]:
# intall miniconda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# update conda
!conda init bash
!conda update -n base -c defaults conda -y

In [ ]:
# install python3.8 with conda
!conda install python=3.8 -y

In [ ]:
!python --version

## change sys.path

In [ ]:
import sys
import pprint

pprint.pprint(sys.path)

# add import path
sys.path.append('/usr/local/lib/python3.8/site-packages')

pprint.pprint(sys.path)

## Githubからソースコード取得

In [ ]:
%cd /content

!git clone https://github.com/microsoft/GLIP.git

# Commits on Jun 27, 2023
%cd /content/GLIP
!git checkout a5f302bfd4c5c67010e29f779e3b0bde94e89985

## ライブラリのインストール

## downgrade cuda and gcc/g++

In [ ]:
!nvcc --version
!gcc -v

In [ ]:
# downgrade cuda
!apt-get update -y
!apt-get --purge remove "*cublas*" "cuda*" "nsight*"
!nvcc --version

!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
!apt-key add /var/cuda-repo-10-2-local-10.2.89-440.33.01/7fa2af80.pub
!apt-get update
!apt-get -y install cuda=10.2.89-1

In [ ]:
# downgrade g++, gcc
!wget -c http://archive.ubuntu.com/ubuntu/pool/universe/g/gcc-5/gcc-5-base_5.5.0-12ubuntu1_amd64.deb
!wget -c http://archive.ubuntu.com/ubuntu/pool/universe/i/isl-0.18/libisl15_0.18-4_amd64.deb
!wget -c http://archive.ubuntu.com/ubuntu/pool/universe/g/gcc-5/cpp-5_5.5.0-12ubuntu1_amd64.deb
!wget -c http://archive.ubuntu.com/ubuntu/pool/universe/g/gcc-5/libasan2_5.5.0-12ubuntu1_amd64.deb
!wget -c http://archive.ubuntu.com/ubuntu/pool/universe/g/gcc-5/libmpx0_5.5.0-12ubuntu1_amd64.deb
!wget -c http://archive.ubuntu.com/ubuntu/pool/universe/g/gcc-5/libgcc-5-dev_5.5.0-12ubuntu1_amd64.deb
!wget -c http://archive.ubuntu.com/ubuntu/pool/universe/g/gcc-5/gcc-5_5.5.0-12ubuntu1_amd64.deb
!wget -c http://archive.ubuntu.com/ubuntu/pool/universe/g/gcc-5/libstdc++-5-dev_5.5.0-12ubuntu1_amd64.deb
!wget -c http://archive.ubuntu.com/ubuntu/pool/universe/g/gcc-5/g++-5_5.5.0-12ubuntu1_amd64.deb

!sudo apt install \
  ./g++-5_5.5.0-12ubuntu1_amd64.deb \
  ./gcc-5_5.5.0-12ubuntu1_amd64.deb \
  ./gcc-5-base_5.5.0-12ubuntu1_amd64.deb \
  ./cpp-5_5.5.0-12ubuntu1_amd64.deb \
  ./libisl15_0.18-4_amd64.deb \
  ./libgcc-5-dev_5.5.0-12ubuntu1_amd64.deb \
  ./libasan2_5.5.0-12ubuntu1_amd64.deb \
  ./libmpx0_5.5.0-12ubuntu1_amd64.deb \
  ./libstdc++-5-dev_5.5.0-12ubuntu1_amd64.deb

In [ ]:
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;

In [ ]:
!nvcc --version
!gcc -v

## install libs

In [ ]:
%cd /content/GLIP

!pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install einops shapely timm yacs tensorboardX ftfy prettytable pymongo opencv-python chardet
!pip install transformers==4.18.0

In [ ]:
%cd /content/GLIP

!python setup.py build develop --user

## ライブラリのインポート

In [ ]:
%cd /content/GLIP

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

import requests
from io import BytesIO
from PIL import Image
import numpy as np
pylab.rcParams['figure.figsize'] = 20, 12
from maskrcnn_benchmark.config import cfg
# from maskrcnn_benchmark.engine.predictor_glip import GLIPDemo

In [ ]:
!sed -i ./maskrcnn_benchmark/engine/predictor_glip.py -e "s/from transformers import AutoTokenizer/# from transformers import AutoTokenizer/g"

cfg.MODEL.LANGUAGE_BACKBONE.TOKENIZER_TYPE = 'clip'
from maskrcnn_benchmark.engine.predictor_glip import GLIPDemo

# 学習済みモデルのセットアップ

In [ ]:
%cd /content/GLIP
!mkdir MODEL

# !wget -c https://penzhanwu2bbs.blob.core.windows.net/data/GLIPv1_Open/models/glip_tiny_model_o365_goldg_cc_sbu.pth \
#       -O MODEL/glip_tiny_model_o365_goldg_cc_sbu.pth

!wget -c https://huggingface.co/harold/GLIP/blob/main/glip_tiny_model_o365_goldg_cc_sbu_lvisbest.pth \
      -O MODEL/glip_tiny_model_o365_goldg_cc_sbu.pth

# Utility関数定義

In [ ]:
def load(url):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    response = requests.get(url)
    pil_image = Image.open(BytesIO(response.content)).convert("RGB")
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

def imshow(img, caption):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")
    plt.figtext(0.5, 0.09, caption, wrap=True, horizontalalignment='center', fontsize=20)

# build model

In [ ]:
%cd /content/GLIP

config_file = "configs/pretrain/glip_Swin_T_O365_GoldG.yaml"
weight_file = "MODEL/glip_tiny_model_o365_goldg_cc_sbu.pth"

cfg.local_rank = 0
cfg.num_gpus = 1
cfg.merge_from_file(config_file)
cfg.merge_from_list(["MODEL.WEIGHT", weight_file])
cfg.merge_from_list(["MODEL.DEVICE", "cuda"])

glip_demo = GLIPDemo(
  cfg,
  min_image_size=800,
  confidence_threshold=0.7,
  show_mask_heatmaps=False,
)

# image, captionの設定

In [ ]:
img_url = 'https://www.pakutaso.com/shared/img/thumb/smIMGL4174_TP_V.jpg' #@param {type:'string'}
caption = 'yellow fruit' #@param {type:'string'}

# Object detection

In [ ]:
glip_demo.color = 128

In [ ]:
image = load(img_url)
result, _ = glip_demo.run_on_web_image(image, caption, 0.5)
imshow(result, caption)